In [80]:
path = '/content/Sale July 24-29th Oct 24 - Copy.xlsx'

In [81]:
import pandas as pd

In [82]:
df = pd.read_excel(path)

In [83]:
df = df[['YEAR', 'MONTH', 'REGION', 'SITE ID','SUPPLYPOINT',
       'DELIVERY ORDER', 'INVOICE DATE',
       'PRODUCT', 'QUANTITY', 'DISCOUNT RATE', 'DISCOUNT AMOUNT',
       'PARTY NAME', 'TSM','PAYMENT MODE']]

In [84]:
# Convert the QUANTITY column to numeric, coercing errors
df['QUANTITY'] = pd.to_numeric(df['QUANTITY'], errors='coerce')
df['DISCOUNT RATE'] = pd.to_numeric(df['DISCOUNT RATE'], errors='coerce')


In [75]:
month_mapping = {
    1: "July",
    2: "August",
    3: "September",
    4: "October",
    5: "November",
    6: "December",
    7: "January",
    8: "February",
    9: "March",
    10: "April",
    11: "May",
    12: "June"
}

# Convert MONTH column to month names using the mapping
df['MONTH'] = df['MONTH'].map(month_mapping)

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7222 entries, 0 to 7221
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   YEAR             7222 non-null   int64  
 1   MONTH            7222 non-null   object 
 2   REGION           7222 non-null   object 
 3   SITE ID          4367 non-null   object 
 4   SUPPLYPOINT      7222 non-null   object 
 5   DELIVERY ORDER   7222 non-null   object 
 6   INVOICE DATE     7222 non-null   object 
 7   PRODUCT          7222 non-null   object 
 8   QUANTITY         7222 non-null   float64
 9   DISCOUNT RATE    7222 non-null   float64
 10  DISCOUNT AMOUNT  7222 non-null   int64  
 11  PARTY NAME       7221 non-null   object 
 12  TSM              6986 non-null   object 
 13  PAYMENT MODE     7222 non-null   object 
dtypes: float64(2), int64(2), object(10)
memory usage: 790.0+ KB


In [97]:
# Step 1: Install Required Libraries
#!pip install transformers pandas torch

# Step 2: Import Libraries
#import pandas as pd
#import torch
from transformers import AutoTokenizer, AutoModel

# Load BERT for simple query understanding
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

# Step 3: Load the Dataset
# data_path = '/path/to/your/dataset.xlsx'  # Replace with your dataset path
# df = pd.read_excel(data_path)

# Step 4: Define Functions to Process Queries and Handle Data
def extract_keywords(query):
    # Tokenize query and get keyword tokens
    inputs = tokenizer(query, return_tensors="pt")
    outputs = model(**inputs)
    keywords = query.lower().split()  # For simplicity, treat all words as keywords
    return keywords

def answer_question(query, df):
    keywords = extract_keywords(query)
    print(keywords)
    # Filter DataFrame by matching keywords to relevant columns
    if 'region' in keywords:
        region = next((word for word in keywords if word in df['REGION'].str.lower().unique()), None)
        if region:
            df = df[df['REGION'].str.lower() == region]

    if 'month' in keywords:
        month = next((word for word in keywords if word in df['MONTH'].str.lower().unique()), None)
        print(month)
        if month:
            df = df[df['MONTH'].str.lower() == month]

    # Example calculations for "sum" or "average" in query
    if 'total' in query.lower() or 'sum' in query.lower():
        result = df['QUANTITY'].astype(float).sum()
    elif 'average' in query.lower():
        result = df['QUANTITY'].astype(float).mean()
    else:
        result = df.head()  # Return a sample of data if no specific aggregation requested

    return result

# Step 5: Test the Query



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [99]:
query = "What is the total QUANTITY for REGION AGRI of Product HSD for the month of July"
print(answer_question(query, df))

['what', 'is', 'the', 'total', 'quantity', 'for', 'region', 'agri', 'of', 'product', 'hsd', 'for', 'the', 'month', 'of', 'july']
july
651000.0


In [85]:
# Step 1: Install Required Libraries
#!pip install transformers pandas torch

# Step 2: Import Libraries
#import pandas as pd
#import torch
#from transformers import AutoTokenizer, AutoModel

# Load BERT for simple query understanding
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

# Step 3: Load the Dataset and Convert 'QUANTITY' and 'MONTH' Columns
# data_path = '/path/to/your/dataset.xlsx'  # Replace with your dataset path
# df = pd.read_excel(data_path)

# Convert QUANTITY column to numeric
df['QUANTITY'] = pd.to_numeric(df['QUANTITY'], errors='coerce')

# Define month mapping dictionary
month_mapping = {
    1: "July",
    2: "August",
    3: "September",
    4: "October",
    5: "November",
    6: "December",
    7: "January",
    8: "February",
    9: "March",
    10: "April",
    11: "May",
    12: "June"
}

# Convert MONTH column to month names using the mapping
df['MONTH'] = df['MONTH'].map(month_mapping)
# Define the mapping dictionary for product names
product_mapping = {
    "High Speed Diesel": "HSD",
    "Premier Motor Gasoline": "PMG",
    # Add more mappings as needed
}

# Apply the mapping to the PRODUCT column
df['PRODUCT'] = df['PRODUCT'].replace(product_mapping)


# Step 4: Define Functions to Process Queries and Handle Data
def extract_keywords(query):
    # Tokenize query and get keyword tokens
    inputs = tokenizer(query, return_tensors="pt")

    outputs = model(**inputs)

    keywords = query.lower().split()     # Basic splitting, can improve with NLP tools
    return keywords

def answer_question(query, df):
    keywords = extract_keywords(query)

    # Start with the full DataFrame and filter down based on keywords
    filtered_df = df.copy()
    print(filtered_df)
    # Check for region in keywords and filter
    if 'region' in keywords:
        region_keyword = next((word for word in keywords if word in filtered_df['REGION'].str.lower().unique()), None)
        print(region_keyword)
        if region_keyword:
            filtered_df = filtered_df[filtered_df['REGION'].str.lower() == region_keyword]

    # Check for month in keywords and filter
    if 'month' in keywords:
        month_keyword = next((word for word in keywords if word in month_mapping.values()), None)
        print(month_keyword)
        if month_keyword:
            filtered_df = filtered_df[filtered_df['MONTH'].str.lower() == month_keyword.lower()]
    if 'product'in keywords:
        product_keyword = next((word for word in keywords if word in filtered_df['PRODUCT'].str.lower().unique()), None)
        print(product_keyword)
        if product_keyword:
            filtered_df = filtered_df[filtered_df['PRODUCT'].str.lower() == product_keyword.lower()]


    # Example calculations for "sum" or "average" in query
    if 'total' in query.lower() or 'sum' in query.lower():
        result = filtered_df['QUANTITY'].sum()
    elif 'average' in query.lower():
        result = filtered_df['QUANTITY'].mean()
    else:
        result = filtered_df.head()  # Return a sample of data if no specific aggregation requested

    return result




/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [88]:
# Step 5: Test the Query
query = "show me the Total quantity sold  for Product HSD for the month of July"
print(answer_question)

<function answer_question at 0x7a0e69531bd0>


In [92]:
query = "What is the total QUANTITY sold Region of  Agri of Product HSD?"
print(answer_question(query, df))

      YEAR      MONTH          REGION      SITE ID SUPPLYPOINT DELIVERY ORDER  \
0     2024       July            Agri          NaN  PORT QASIM     ADVS000001   
1     2024       July            Agri          NaN  PORT QASIM     ADVS000001   
2     2024       July            Agri          NaN  PORT QASIM     ADVS000002   
3     2024       July            Agri          NaN  PORT QASIM     ADVS000003   
4     2024       July            Agri          NaN  PORT QASIM     ADVS000003   
...    ...        ...             ...          ...         ...            ...   
7217  2024  September    Retail North  NOR_KPK_021    TARUJABA     HOOS003993   
7218  2024  September    Retail North  NOR_AKG_027      SIHALA     HOOS003994   
7219  2024  September    Retail North  NOR_AKG_027      SIHALA     HOOS003995   
7220  2024  September  Retail Central  CEN_SHK_006    MACHIKEE     HOOS003996   
7221  2024  September  Retail Central  CEN_SHK_006    MACHIKEE     HOOS003996   

               INVOICE DATE